<a href="https://colab.research.google.com/github/hamletbatista/state-of-search/blob/main/Quality_Content_at_Scale_Through_Automated_Text_Summarization_of_UGC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Quality Content at Scale Through Automated Text Summarization of UGC

1. Downloading Amazon Reviews
2. Deduplication
3. FAQs Generation



In [ ]:
amazon_reviews_dataset = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Wireless_v1_00.tsv.gz" #@param ["https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Wireless_v1_00.tsv.gz", "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Watches_v1_00.tsv.gz", "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz","https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_DVD_v1_00.tsv.gz","https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_v1_00.tsv.gz","https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Toys_v1_00.tsv.gz", "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Tools_v1_00.tsv.gz","https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Sports_v1_00.tsv.gz","https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Software_v1_00.tsv.gz", "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Shoes_v1_00.tsv.gz"]


## Downloading Amazong Reviews

https://s3.amazonaws.com/amazon-reviews-pds/readme.html

https://s3.amazonaws.com/amazon-reviews-pds/tsv/index.txt




In [ ]:
%%capture
!pip install wget


In [ ]:
import wget

filename = wget.download(amazon_reviews_dataset)

In [ ]:
!ls

amazon_reviews_us_Wireless_v1_00.tsv.gz  sample_data


In [ ]:
#!head amazon_reviews_us_Wireless_v1_00.tsv

marketplace	customer_id	review_id	product_id	product_parent	product_title	product_category	star_rating	helpful_votes	total_votes	vine	verified_purchase	review_headline	review_body	review_date
US	16414143	R3W4P9UBGNGH1U	B00YL0EKWE	852431543	LG G4 Case Hard Transparent Slim Clear Cover for LG G4	Wireless	2	1	3	N	Y	Looks good, functions meh	2 issues  -  Once I turned on the circle apps and installed this case,  my battery drained twice as fast as usual.  I ended up turning off the circle apps, which kind of makes the case just a case...  with a hole in it.  Second,  the wireless charging doesn't work.  I have a Motorola 360 watch and a Qi charging pad. The watch charges fine but this case doesn't. But hey, it looks nice.	2015-08-31
US	50800750	R15V54KBMTQWAY	B00XK95RPQ	516894650	Selfie Stick Fiblastiq&trade; Extendable Wireless Bluetooth Selfie Stick with built-in Bluetooth Adjustable Phone Holder	Wireless	4	0	0	N	N	A fun little gadget	I’m embarrassed to admit that until recently, I have 

In [ ]:
#filename

In [ ]:
!gunzip *.tsv.gz

In [ ]:
!ls -l

total 4099676
-rw-r--r-- 1 root root 4198056580 Oct 11 04:03 amazon_reviews_us_Wireless_v1_00.tsv
drwxr-xr-x 1 root root       4096 Oct  5 16:31 sample_data


In [ ]:
import pandas as pd
from glob import glob

files = glob("*.tsv")

first = files[0]

df = pd.read_table(first, error_bad_lines=False)

b'Skipping line 51163: expected 15 fields, saw 22\n'
b'Skipping line 77018: expected 15 fields, saw 22\nSkipping line 97528: expected 15 fields, saw 22\n'
b'Skipping line 145503: expected 15 fields, saw 22\nSkipping line 172172: expected 15 fields, saw 22\nSkipping line 185136: expected 15 fields, saw 22\nSkipping line 187592: expected 15 fields, saw 22\n'
b'Skipping line 222158: expected 15 fields, saw 22\nSkipping line 232048: expected 15 fields, saw 22\nSkipping line 232320: expected 15 fields, saw 22\nSkipping line 234653: expected 15 fields, saw 22\nSkipping line 257210: expected 15 fields, saw 22\nSkipping line 261368: expected 15 fields, saw 22\n'
b'Skipping line 262843: expected 15 fields, saw 22\nSkipping line 274082: expected 15 fields, saw 22\nSkipping line 276172: expected 15 fields, saw 22\nSkipping line 327076: expected 15 fields, saw 22\n'
b'Skipping line 328675: expected 15 fields, saw 22\nSkipping line 366744: expected 15 fields, saw 22\nSkipping line 380897: expected 

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8991589 entries, 0 to 8991588
Data columns (total 15 columns):
 #   Column             Dtype  
---  ------             -----  
 0   marketplace        object 
 1   customer_id        int64  
 2   review_id          object 
 3   product_id         object 
 4   product_parent     int64  
 5   product_title      object 
 6   product_category   object 
 7   star_rating        object 
 8   helpful_votes      float64
 9   total_votes        float64
 10  vine               object 
 11  verified_purchase  object 
 12  review_headline    object 
 13  review_body        object 
 14  review_date        object 
dtypes: float64(2), int64(2), object(11)
memory usage: 1.0+ GB


In [ ]:
df.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,16414143,R3W4P9UBGNGH1U,B00YL0EKWE,852431543,LG G4 Case Hard Transparent Slim Clear Cover f...,Wireless,2,1.0,3.0,N,Y,"Looks good, functions meh",2 issues - Once I turned on the circle apps ...,2015-08-31
1,US,50800750,R15V54KBMTQWAY,B00XK95RPQ,516894650,Selfie Stick Fiblastiq&trade; Extendable Wirel...,Wireless,4,0.0,0.0,N,N,A fun little gadget,"I’m embarrassed to admit that until recently, ...",2015-08-31
2,US,15184378,RY8I449HNXSVF,B00SXRXUKO,984297154,Tribe AB40 Water Resistant Sports Armband with...,Wireless,5,0.0,0.0,N,Y,Five Stars,Fits iPhone 6 well,2015-08-31
3,US,10203548,R18TLJYCKJFLSR,B009V5X1CE,279912704,RAVPower® Element 10400mAh External Battery US...,Wireless,5,0.0,0.0,N,Y,Great charger,Great charger. I easily get 3+ charges on a S...,2015-08-31
4,US,488280,R1NK26SWS53B8Q,B00D93OVF0,662791300,Fosmon Micro USB Value Pack Bundle for Samsung...,Wireless,5,0.0,0.0,N,Y,Five Stars,Great for the price :-),2015-08-31


In [ ]:
df.query("star_rating == 5 & total_votes > 50.0 & verified_purchase == 'Y'")


,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
3164,US,16980808,R2EE2TR4MRDV0U,B00Y1Z87UU,956867660,IBESTWIN Li-ion Replacement Battery for Samsun...,Wireless,5,53.0,59.0,N,Y,Does it get any better than a 10000 mah battery?,A great battery for a great phone. Some of us ...,2015-08-31
5049,US,35621482,R1D3NR5GREEXXJ,B013F8C9X4,561786475,"Galaxy Note 5 Case, Verus [Verge][Parent] - [B...",Wireless,5,57.0,58.0,N,Y,Great case for a nice price,I've been a big fan of a competing case brand ...,2015-08-31
5185,US,44137838,R2WYKBQS8OR08O,B010GYYSU2,518226514,"Z-Edge 2.4- inch 135¡ã View Angle, Full HD 108...",Wireless,5,168.0,176.0,N,Y,PERFECT DASH CAM !,"I returned my garmin dashcam, and change it wi...",2015-08-31
6856,US,39442138,R6IEQ9100RX0O,B010R8CE4O,782031320,Ringke [FUSION] iPhone SE Case Crystal Clear P...,Wireless,5,49.0,57.0,N,Y,This is a pretty awesome case,This is a pretty awesome case. Wasn't expecti...,2015-08-31
9101,US,30337045,R3VM9PM5RHZW3C,B0133Z8U28,276707711,"Apple Watch Band, JETech 38mm Milanese Loop St...",Wireless,5,93.0,98.0,N,Y,Happy wife happy life,I purchased as a gift for my wife and she love...,2015-08-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8991089,US,51942536,R21P3L38CHEFED,B00003OPEU,217309530,Garmin eMap Hiking GPS,Wireless,5,85.0,86.0,N,Y,Great product - Beware of hidden costs though,"Don't get me wrong, I love this GPS. For me a ...",2000-10-24
8991095,US,49597918,R5BY4DTFKCQAH,B00004XOM5,446786010,Unwired Technology UFR3082 2-Mile 14-Channel F...,Wireless,5,84.0,92.0,N,Y,"Good product, good price, everything works",Some tests with a GPS on open field and city.<...,2000-10-22
8991098,US,49597918,R20NTLWYCXJXA0,B00004XOM5,446786010,Unwired Technology UFR3082 2-Mile 14-Channel F...,Wireless,5,68.0,68.0,N,Y,"Fast delivery, very good product",I received a pair as said. <br />I am not a FR...,2000-10-20
8991150,US,49355663,R4UIFJSVRSN6A,B00000K2YR,122734261,Midland 75-822 40-Channel CB Radio,Wireless,5,190.0,199.0,N,Y,compact and versatile,"This is a very compact, well designed, and ver...",2000-09-13


Select 5-star rating batteries with at least 50 votes

In [ ]:
df.query("star_rating == 5 & verified_purchase == 'Y' & total_votes > 50.0 & product_title.str.contains('Battery')", engine='python' )

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
3164,US,16980808,R2EE2TR4MRDV0U,B00Y1Z87UU,956867660,IBESTWIN Li-ion Replacement Battery for Samsun...,Wireless,5,53.0,59.0,N,Y,Does it get any better than a 10000 mah battery?,A great battery for a great phone. Some of us ...,2015-08-31
167567,US,2263238,RT7P8L8Z5RNY5,B00XC2FA1C,925404577,LEMFO Bluetooth Leather Smart Watch with Camer...,Wireless,5,82.0,84.0,N,Y,This is the best Smartwatch i never seen,This is the best Smartwatch i never seen... I ...,2015-08-18
168866,US,217449,R132610NMNJYXJ,B00WABHBX6,160367424,ScandiTech 3.8V 1440 mAh Li-ion Replacement Ba...,Wireless,5,70.0,73.0,N,Y,Took me 5 mins,Searched so long for a good working battery an...,2015-08-18
249320,US,44281042,R1HD4CKG05AXXC,B00XC2FA1C,925404577,LEMFO Bluetooth Leather Smart Watch with Camer...,Wireless,5,89.0,92.0,N,Y,This is not my first smart watch but it is the...,This is not my first smart watch but it is the...,2015-08-11
373107,US,29334033,R2V5CGSR8XDAAO,B00UVQSZU0,221813350,New 1560 mAh BELTRON Replacement Battery for i...,Wireless,5,48.0,51.0,N,Y,I feel like this is and was a scam and Amazon ...,I feel like this is and was a scam and Amazon ...,2015-08-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8718915,US,52196389,R74PJ2982H7C8,B002ZG8BQM,440584380,Motorola DROID and DROID II 1300mah Standard ...,Wireless,5,89.0,95.0,N,Y,genuine Motorola battery,I bought this battery direct from Amazon. I d...,2010-03-05
8762893,US,37455134,R1UH99IKOJLN3H,B001FVPU9O,418237493,Skiva - Protective Case & Extended Battery for...,Wireless,5,67.0,68.0,N,Y,Otterbox cases are like insurance for the unin...,I purchased the OtterBox Defender Case for my ...,2009-10-23
8800157,US,13430525,R26IS5PYPHAWZP,B001FVPU9O,418237493,Skiva - Protective Case & Extended Battery for...,Wireless,5,243.0,255.0,N,Y,Do you hear an ECHO?!?!,I am not usually one to leave review comments....,2009-05-31
8846552,US,52453617,R3CLDKHH3BP3R2,B0013G8PTS,26557174,New Trent: iTorch 5200mAh Ultra Portable USB P...,Wireless,5,315.0,333.0,N,Y,Works for Iphone and Kindle too,I love my Iphone 3G. It seems every day I fin...,2008-11-15


Combine the reviews to one text.

In [ ]:
reviews = df.query("star_rating == 5 & verified_purchase == 'Y' & total_votes > 50.0 & product_title.str.contains('Battery')", engine='python' )["review_body"].tolist()



In [ ]:
reviews[:20]

["A great battery for a great phone. Some of us power users need more than the standard and this is perfect. There isn't a bigger battery on the market. A full charge will last you at least three days. It's going to take a couple charging cycles for the phone to recognize the battery fully and you won't have to panic and rush for a charger at one percent either. 1% of a 10000 mah battery is another 5 hrs. Just try not to constantly run the battery down to 0.",
 'This is the best Smartwatch i never seen... I recommend everyone who have a iphone to buy this Smartwatch......You can see everything notifications text messages you can answer phone calls . you can take picture from the watch you can video record too its awesome',
 "Searched so long for a good working battery and believe me I've found one! This one! My battery was swollen to the point where my screen was raising and my battery was dying so fast but this battery got rid of those issues!<br />It came fully charged and having onl

Remove unnecesary tags

In [ ]:
import re

#remove video reviews tags
text = "\n".join([re.sub("<br />|\[\[VIDEOID:.+\]\]", "", x) for x in reviews if isinstance(x, str)])


In [ ]:
text

'A great battery for a great phone. Some of us power users need more than the standard and this is perfect. There isn\'t a bigger battery on the market. A full charge will last you at least three days. It\'s going to take a couple charging cycles for the phone to recognize the battery fully and you won\'t have to panic and rush for a charger at one percent either. 1% of a 10000 mah battery is another 5 hrs. Just try not to constantly run the battery down to 0.\nThis is the best Smartwatch i never seen... I recommend everyone who have a iphone to buy this Smartwatch......You can see everything notifications text messages you can answer phone calls . you can take picture from the watch you can video record too its awesome\nSearched so long for a good working battery and believe me I\'ve found one! This one! My battery was swollen to the point where my screen was raising and my battery was dying so fast but this battery got rid of those issues!It came fully charged and having only one cyc

In [ ]:
len(text)

545363

## Deduplication

Filter duplicate sentences using Sentence Embeddings

https://github.com/UKPLab/sentence-transformers


https://github.com/huggingface/transformers/issues/876


https://towardsdatascience.com/deduplication-deduplication-1d1414ffb4d2


In [ ]:
%%capture
!pip install sentence-transformers


In [ ]:
#from sentence_transformers import SentenceTransformer, util
#model = SentenceTransformer('distilbert-base-nli-mean-tokens')
#https://github.com/UKPLab/sentence-transformers/issues/485
#model = SentenceTransformer('xlm-r-100langs-bert-base-nli-stsb-mean-tokens')


Example code

In [ ]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')

# Two lists of sentences
sentences1 = ['The cat sits outside',
             'A man is playing guitar',
             'The new movie is awesome']

sentences2 = ['The dog plays in the garden',
              'A woman watches TV',
              'The new movie is so great']

#Compute embedding for both lists
embeddings1 = model.encode(sentences1, convert_to_tensor=True)
embeddings2 = model.encode(sentences2, convert_to_tensor=True)

#Compute cosine-similarits
cosine_scores = util.pytorch_cos_sim(embeddings1, embeddings2)

#Output the pairs with their score
for i in range(len(sentences1)):
    print("{} \t\t {} \t\t Score: {:.4f}".format(sentences1[i], sentences2[i], cosine_scores[i][i]))


100%|██████████| 245M/245M [00:32<00:00, 7.58MB/s]


The cat sits outside 		 The dog plays in the garden 		 Score: 0.2166
A man is playing guitar 		 A woman watches TV 		 Score: -0.0146
The new movie is awesome 		 The new movie is so great 		 Score: 0.9816


Split text into sentences

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
import nltk.data

tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [ ]:
sentences = tokenizer.tokenize(text)

In [ ]:
len(sentences)

4664

In [ ]:
sentences[:10]

['A great battery for a great phone.',
 'Some of us power users need more than the standard and this is perfect.',
 "There isn't a bigger battery on the market.",
 'A full charge will last you at least three days.',
 "It's going to take a couple charging cycles for the phone to recognize the battery fully and you won't have to panic and rush for a charger at one percent either.",
 '1% of a 10000 mah battery is another 5 hrs.',
 'Just try not to constantly run the battery down to 0.',
 'This is the best Smartwatch i never seen...',
 'I recommend everyone who have a iphone to buy this Smartwatch......You can see everything notifications text messages you can answer phone calls .',
 "you can take picture from the watch you can video record too its awesome\nSearched so long for a good working battery and believe me I've found one!"]

In [ ]:
duplicate_indexes = list()

max = int(len(sentences) /2 )

new_sentences = sentences.copy()

# Two lists of sentences
sentences1 = sentences[:max]

sentences2 = sentences[max:]

#Compute embedding for both lists
embeddings1 = model.encode(sentences1, convert_to_tensor=True)
embeddings2 = model.encode(sentences2, convert_to_tensor=True)

#Compute cosine-similarits
cosine_scores = util.pytorch_cos_sim(embeddings1, embeddings2)

#Output the pairs with their score
for i in range(len(sentences1)):
    score = cosine_scores[i][i]

    if score > 0.6:
        print(f"{i}. {sentences1[i]} <> {sentences2[i]} \nScore: {score:.4f}")
        new_sentences[i] = "" #blank sentence


179. It works through the headphone input so you don't need to worry about Bluetooth pairing or batteries, which is why I bought this model.It's lightweight and easy to expand and collapse. <> then it also has a regular USB port that you use to plug your phone/ipod/kindle/psvita or anything that can charge using a regular USB port and it is just like it plugged into a regular wall charger.The 1000mAh output means that it should take roughly the same amount of time to charge your device as it would take when it is plugged into the wall, and the 2600mA capacity should be enough capacity to be able to fully charge your device at least one time (just divide 2600 by your battery capacity and that is how many times you can fully recharge your battery from this battery without having to recharge this battery - for my droid3 battery is 1500mA capacity, so I can recharge it fully 2600/1500=1.73 times, my ipod4g battery capacity is 930mA so I can recharge it 2600/930=2.8 times).It is small enoug

In [ ]:
import pickle

In [ ]:
with open("sentences.pkl", "wb") as f:
  pickle.dump(sentence_embeddings, f)

In [ ]:
with open("sentences_raw.pkl", "wb") as f:
  pickle.dump(sentences, f)

In [ ]:
!ls

amazon_reviews_us_Wireless_v1_00.tsv  sentences.pkl
sample_data			      sentences_raw.pkl


In [ ]:
with open("sentences.pkl", "rb") as f:
  sentence_embeddings = pickle.load(f)

In [ ]:
with open("sentences_raw.pkl", "rb") as f:
  sentences = pickle.load(f)

#FAQ Generation

In [ ]:
!gdown -O  t5_que_gen.zip --id 1vhsDOW9wUUO83IQasTPlkxb82yxmMH-V
!unzip t5_que_gen.zip

Downloading...
From: https://drive.google.com/uc?id=1vhsDOW9wUUO83IQasTPlkxb82yxmMH-V
To: /content/t5_que_gen.zip
1.65GB [00:16, 98.2MB/s]
Archive:  t5_que_gen.zip
   creating: t5_que_gen_model/
   creating: t5_que_gen_model/t5_base_tok_que_gen/
  inflating: t5_que_gen_model/t5_base_tok_que_gen/spiece.model  
 extracting: t5_que_gen_model/t5_base_tok_que_gen/added_tokens.json  
 extracting: t5_que_gen_model/t5_base_tok_que_gen/tokenizer_config.json  
  inflating: t5_que_gen_model/t5_base_tok_que_gen/special_tokens_map.json  
   creating: t5_que_gen_model/t5_base_que_gen/
  inflating: t5_que_gen_model/t5_base_que_gen/config.json  
  inflating: t5_que_gen_model/t5_base_que_gen/pytorch_model.bin  
 extracting: t5_que_gen_model/logs.zip  
   creating: t5_ans_gen_model/
   creating: t5_ans_gen_model/t5_base_tok_ans_gen/
  inflating: t5_ans_gen_model/t5_base_tok_ans_gen/spiece.model  
  inflating: t5_ans_gen_model/t5_base_tok_ans_gen/added_tokens.json  
 extracting: t5_ans_gen_model/t5_base_

In [ ]:
import argparse
import glob
import os
import json
import time
import logging
import random
from itertools import chain
from string import punctuation

import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
class QueGenerator():
  def __init__(self):
    self.que_model = T5ForConditionalGeneration.from_pretrained('./t5_que_gen_model/t5_base_que_gen/')
    self.ans_model = T5ForConditionalGeneration.from_pretrained('./t5_ans_gen_model/t5_base_ans_gen/')

    self.que_tokenizer = T5Tokenizer.from_pretrained('./t5_que_gen_model/t5_base_tok_que_gen/')
    self.ans_tokenizer = T5Tokenizer.from_pretrained('./t5_ans_gen_model/t5_base_tok_ans_gen/')
    
    self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    self.que_model = self.que_model.to(self.device)
    self.ans_model = self.ans_model.to(self.device)
  
  def generate(self, text):
    answers = self._get_answers(text)
    questions = self._get_questions(text, answers)
    output = [{'answer': ans, 'question': que} for ans, que in zip(answers, questions)]
    return output
  
  def _get_answers(self, text):
    # split into sentences
    sents = sent_tokenize(text)

    examples = []
    for i in range(len(sents)):
      input_ = ""
      for j, sent in enumerate(sents):
        if i == j:
            sent = "[HL] %s [HL]" % sent
        input_ = "%s %s" % (input_, sent)
        input_ = input_.strip()
      input_ = input_ + " </s>"
      examples.append(input_)
    
    batch = self.ans_tokenizer.batch_encode_plus(examples, max_length=512, pad_to_max_length=True, return_tensors="pt")
    with torch.no_grad():
      outs = self.ans_model.generate(input_ids=batch['input_ids'].to(self.device), 
                                attention_mask=batch['attention_mask'].to(self.device), 
                                max_length=32,
                                # do_sample=False,
                                # num_beams = 4,
                                )
    dec = [self.ans_tokenizer.decode(ids, skip_special_tokens=False) for ids in outs]
    answers = [item.split('[SEP]') for item in dec]
    answers = chain(*answers)
    answers = [ans.strip() for ans in answers if ans != ' ']
    return answers
  
  def _get_questions(self, text, answers):
    examples = []
    for ans in answers:
      input_text = "%s [SEP] %s </s>" % (ans, text)
      examples.append(input_text)
    
    batch = self.que_tokenizer.batch_encode_plus(examples, max_length=512, pad_to_max_length=True, return_tensors="pt")
    with torch.no_grad():
      outs = self.que_model.generate(input_ids=batch['input_ids'].to(self.device), 
                                attention_mask=batch['attention_mask'].to(self.device), 
                                max_length=32,
                                num_beams = 4)
    dec = [self.que_tokenizer.decode(ids, skip_special_tokens=False) for ids in outs]
    return dec

In [ ]:
que_generator = QueGenerator()

In [ ]:
new_text = " ".join(new_sentences)

In [ ]:
len(new_text)

539805

In [ ]:
faqs = que_generator.generate(new_text[:2000])

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1773: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.6/dist-packages/transformers/tokenization_t5.py:185: UserWarning: This sequ

In [ ]:
faqs

[{'answer': 'A great battery for a great phone',
  'question': 'How does the battery compare to other smartwatches?'},
 {'answer': 'Some of us power users need more than the standard',
  'question': 'Why is this a great battery for a smartwatch?'},
 {'answer': 'bigger',
  'question': 'Is the leather band of the T-Mobile Smartwatch a little bigger or smaller than the standard one?'},
 {'answer': 'three days', 'question': 'How long does a full charge last?'},
 {'answer': 'a couple charging cycles',
  'question': 'How long does it take for the phone to recognize the battery?'},
 {'answer': '1',
  'question': 'What percent of a 10000 mah battery is another 5 hrs?'},
 {'answer': 'Try not to constantly run the battery down to 0.',
  'question': 'How long does it take a 10000 mah battery to recognize a smartwatch?'},
 {'answer': 'This is the best Smartwatch i never seen',
  'question': 'How does the battery compare to other smartwatches out there?'},
 {'answer': 'you can take picture from the

Let's generate the FAQPage schema

In [ ]:
faqpage_template="""<script type="application/ld+json">

{

"@context": "https://schema.org",

"@type": "FAQPage",

"mainEntity": [

{% for faq in faqs %}

{

"@type": "Question",

"name": {{faq.question|tojson}},

"acceptedAnswer": {

"@type": "Answer",

"text": {{faq.answer|tojson}}

}

}{{ "," if not loop.last }}

{% endfor %}

]

}

</script>"""

In [ ]:
from jinja2 import Template

In [ ]:
template=Template(faqpage_template)

faqpage_output=template.render(faqs=faqs)

In [ ]:
print(faqpage_output)

<script type="application/ld+json">

{

"@context": "https://schema.org",

"@type": "FAQPage",

"mainEntity": [



{

"@type": "Question",

"name": "How does the battery compare to other smartwatches?",

"acceptedAnswer": {

"@type": "Answer",

"text": "A great battery for a great phone"

}

},



{

"@type": "Question",

"name": "Why is this a great battery for a smartwatch?",

"acceptedAnswer": {

"@type": "Answer",

"text": "Some of us power users need more than the standard"

}

},



{

"@type": "Question",

"name": "Is the leather band of the T-Mobile Smartwatch a little bigger or smaller than the standard one?",

"acceptedAnswer": {

"@type": "Answer",

"text": "bigger"

}

},



{

"@type": "Question",

"name": "How long does a full charge last?",

"acceptedAnswer": {

"@type": "Answer",

"text": "three days"

}

},



{

"@type": "Question",

"name": "How long does it take for the phone to recognize the battery?",

"acceptedAnswer": {

"@type": "Answer",

"text": "a couple cha

In [ ]:
import pickle

In [ ]:
with open("new_text.pkl", "wb") as f:
  pickle.dump(new_text, f)

In [ ]:
with open("new_text.pkl", "rb") as f:
  new_text=pickle.load(f)